In [ ]:
%%bash
DATA_DIR=/tmp/asd-diagnosis

if [ ! -d $DATA_DIR ]; then
  mkdir -p $DATA_DIR
fi

wget https://minio.131.154.99.37.myip.cloud.infn.it/hackathon-data/asd-diagnosis/Harmonized_structural_features.csv -O $DATA_DIR/Harmonized_structural_features.csv &> .log
wget https://minio.131.154.99.37.myip.cloud.infn.it/hackathon-data/asd-diagnosis/Harmonized_functional_features.csv -O $DATA_DIR/Harmonized_functional_features.csv &> .log
wget https://minio.131.154.99.37.myip.cloud.infn.it/hackathon-data/asd-diagnosis/dict.csv -O $DATA_DIR/dict.csv &> .log

ls -lrth $DATA_DIR/*

In [ ]:
import logging, os
logging.disable(logging.WARNING)
# https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

import warnings
# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Autism Spectrum Disorders (ASD) diagnosis combining structural and functional Magnetic Resonance Imaging and Radiomics¶

For an introduction to Autism Spectrum Disorders (ASD) and to the dataset of this ML example please refer to the [first notebook](sMRI_fMRI_sep.ipynb). In that notebook structural and functional MRI features datasets were used separately. While here we are going to combine those datasets and evaluate if the ML model predictions have improved. 

# Import dataframe and normalize with RobustScaler: DF_normalized
Pre-processing is the same as the first notebook.

In [ ]:
import pandas as pd

path_to_data = '/tmp/asd-diagnosis/'

DF_struct = pd.read_csv(os.path.join(path_to_data,'Harmonized_structural_features.csv'))
DF_funct  = pd.read_csv(os.path.join(path_to_data,'Harmonized_functional_features.csv'))

In [ ]:
DF_struct = DF_struct.set_index('FILE_ID')
DF_funct = DF_funct.set_index('FILE_ID')
DF_funct= DF_funct.drop(['SITE', 'Database_Abide', 'AGE_AT_SCAN', 'DX_GROUP'], axis =1)
DF_merge = DF_struct.join(DF_funct,how='inner')
DF_merge = DF_merge.reset_index()
DF_merge.loc[DF_merge.DX_GROUP == -1, 'DX_GROUP'] = 0
DF_merge

In [ ]:
DF_normalized = DF_merge.drop(['SITE', 'Database_Abide', 'AGE_AT_SCAN', 'FILE_ID'], axis =1)

In [ ]:
from sklearn.preprocessing import RobustScaler

RS_instance = RobustScaler()
DF_normalized.iloc[:,1:] = RS_instance.fit_transform(DF_normalized.iloc[:,1:].to_numpy())
DF_normalized

# ASD/TD classification using both structural and functional features

First, you have to define the model architecture. To implement a joint fusion approach you have to use as input both structural and functional features to 2 different neural networks.

### HINT: you can copy the networks of the other notebook

Then you have to find a way to merge these two networks.

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

import logging
tf.get_logger().setLevel(logging.ERROR)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Concatenate, Dropout
from tensorflow.keras.regularizers import l1

In [ ]:
def structural_model():
    """ This function returns a model ...
    """
    
    return model

def functional_model():
    """ This function returns a model ...
    """
    
    return model

def joint_model(model_1, model_2):
    """ This function combines the output of two keras models
    model_1
    model_2

    returns: a new model which combines model_1 and model_2 by adding 3 dense layers
    """
    
    return Model(inputs = ..., outputs = ...)

## Callback

A callback is an action passed to the model fit function which is performend while training the neural network. These actions allow you to modify certain parameters when a specific condition is met while training. These actions may be implemented before or after an epoch or batch is processed, when training begins or ends and when a specific condition is met. Callbacks can help with tasks like saving the model, reducing learning rates, stopping training early, or logging performance metrics. Here we implement two actions:

* EarlyStopping
* ReduceLROnPlateau

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

early_stop = EarlyStopping(
    monitor              = ... , # https://keras.io/api/callbacks/early_stopping/ 
    patience             = ... , 
    restore_best_weights = ...)

reduce_on_plateau = ReduceLROnPlateau(
    monitor   = ... , # refer to -> https://keras.io/api/callbacks/reduce_lr_on_plateau/
    factor    = ... ,
    patience  = ... ,
    verbose   = ... ,
    mode      = ... ,
    min_delta = ... ,
    cooldown  = ... ,
    min_lr    = ... )

# Cross-validation setting

In [ ]:
import numpy as np

acc = []
AUC = []
shap_values_per_cv_s =[]
shap_values_per_cv_f =[]
var_f = []
var_s = []
np.random.seed(1) # Reproducibility
n_cv = 10 #NUMERO DI KFOLD
rs_=13 
tprs = []
aucs = []
interp_fpr = np.linspace(0, 1, 100)

In [ ]:
import time

from tensorflow.keras.optimizers import SGD #stochastic gradient descent
from tensorflow.keras.backend import clear_session

# import sklearn 
from sklearn.model_selection import StratifiedKFold #train_test_split, KFold
from sklearn.metrics import roc_curve, auc

import shap
import matplotlib.pyplot as plt

In [ ]:
cv = StratifiedKFold(n_splits=n_cv, shuffle=True, random_state=rs_) # Set random state

for train_index, test_index in cv.split(DF_normalized.iloc[:, 1:], DF_normalized.iloc[:, 0]):
    start = time.time()
    df_train, df_test = DF_normalized.iloc[train_index, :], DF_normalized.iloc[test_index, :]

    X_train_struct, X_test_struct = df_train.iloc[:, 1:222], df_test.iloc[:, 1:222]
    X_train_funct, X_test_funct = df_train.iloc[:, 222:], df_test.iloc[:, 222:]
    y_train, y_test = df_train.iloc[:, 0], df_test.iloc[:, 0]

        #load the model
    clear_session()
    mod_1 = structural_model
    mod_2 = functional_model
    model_joint = joint_model(mod_1(), mod_2())

    # Compile the model
    model_joint.compile( # refer to https://keras.io/api/models/model_training_apis/
        optimizer = ... , 
        loss      = ... , 
        metrics   = ...)

    # Fit data to model
    history = model_joint.fit(
        ... ,# refer to https://keras.io/api/models/model_training_apis/
        ... ,
        batch_size            = ... ,
        epochs                = ... ,
        verbose               = ... ,
        validation_data       = ... ,
        validation_batch_size = ... ,
        callbacks             = ...)
    
    #Train and validation accuracy
    plt.figure(figsize=(5, 5))
    plt.subplot(2, 2, 1)
    plt.plot(history.history['accuracy'], label='Training ')
    plt.plot(history.history['val_accuracy'], label='Validation ')
    plt.legend(loc='lower right')
    plt.title('Accuracy')
    #Train and validation loss
    plt.subplot(2, 2, 2)
    plt.plot(history.history['loss'], label='Training ')
    plt.plot(history.history['val_loss'], label='Validation ')
    plt.legend(loc='upper right')
    plt.title(' Loss')
    plt.show()
    
    # prepare for SHAP
    X_train_struct_SHAP = X_train_struct.to_numpy()
    X_test_struct_SHAP = X_test_struct.to_numpy()
    X_train_funct_SHAP = X_train_funct.to_numpy()
    X_test_funct_SHAP = X_test_funct.to_numpy()

    # Use SHAP to explain predictions
    explainer = shap.GradientExplainer(model_joint, [X_train_struct_SHAP, X_train_funct_SHAP])
    shap_values = explainer.shap_values([X_test_struct_SHAP, X_test_funct_SHAP])
    
    #  SHAP information per fold
    shap_values_per_cv_s.append(shap_values[0]) #  221 features, append an array with dims 138/139,  221, 1
    shap_values_per_cv_f.append(shap_values[1]) # 5253 features, append an array with dims 138/139, 5253, 1

    print("len(shap_values[0]) ",len(shap_values[0]))
    print("shap_values[0].shape ",shap_values[0].shape)
    print("len(shap_values[1]) ",len(shap_values[1]))
    print("shap_values[1].shape ",shap_values[1].shape)

    #print(X_test_struct.shape)
        ###########################################################
    _, val_acc = model_joint.evaluate((X_test_struct, X_test_funct), y_test, verbose=0)
    acc.append(val_acc)

        #Compute Receiver operating characteristic (ROC)
    i=0
    preds = model_joint.predict((X_test_struct, X_test_funct), verbose=1)
    fpr, tpr, _ = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
    interp_tpr = np.interp(interp_fpr, fpr, tpr)
    tprs.append(interp_tpr)
    AUC.append(roc_auc)
    i += 1
    print('---------------------AUC------------------', roc_auc)
    end = time.time()
    print('----------------------T-------------------', end - start)
    print("")

In [ ]:
plt.figure()
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
      label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(interp_fpr, mean_tpr)
std_auc = np.std(AUC)
plt.plot(interp_fpr, mean_tpr, color='b',
        label=f'Mean ROC (AUC = {mean_auc:.2f} $\pm$ {std_auc:.2f})',
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(interp_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Joint Fusion model',fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
# == Provide average scores ==
print(len(AUC))
print(f'AUC:{np.mean(AUC):.4f} (+- {np.std(AUC):.4f})')
print(f'accuracy: {np.mean(acc):.4f} (+- {np.std(acc):.4f})')

In [ ]:
# Establish lists to keep average Shap values
average_shap_values_s = []
average_shap_values_f = []

for i in range(0, len(AUC)):
    df_per_obs = shap_values_per_cv_s[i].copy()
    df_per_obs = np.absolute(df_per_obs)
    average_shap_values_s.append(df_per_obs.mean(axis=0))

    df_per_f = shap_values_per_cv_f[i].copy()
    df_per_f = np.absolute(df_per_f)
    average_shap_values_f.append(df_per_f.mean(axis=0))

fold_s = np.transpose(np.array(average_shap_values_s)[...,0])
fold_f = np.transpose(np.array(average_shap_values_f)[...,0])

# SHAP values Joint model: NORMALIZATION SHAP VALUES 

Here, we normalize SHAP values separatly for structural and functional model. The SHAP values for each cross-validation fold are processed as follows:

1) They are transformed into a DataFrame
2) Then the SHAP values are weighted and expressed in percentage, taking into account the fact that the number of functional features is higher with respect do the structural one. Without this weight the SHAP values of structural features would be very much higher with respect to the functional one.
3) Finally, the SHAP values are concatenated and sorted according to their values.

In [ ]:
fold_s = pd.DataFrame.from_dict(fold_s)
fold_f = pd.DataFrame.from_dict(fold_f)

In [ ]:
# Definition of the weighting factor
s = 221/((221 + 5253))  
f = 5253/((221 + 5253))

In [ ]:
fold_s_n = (fold_s/fold_s.sum(axis=0))*s*100
plot = fold_s_n.mean(axis=1).values
struct_SHAP = pd.DataFrame({"SHAP_values": plot}, index = DF_normalized.iloc[:, 1:222].columns )
struct_SHAP['std']=fold_s_n.std(axis=1).values
struct_SHAP = struct_SHAP.sort_values(by="SHAP_values", ascending=False)
struct_SHAP

In [ ]:
norm = fold_f.sum(axis=0)
fold_f_n = (fold_f/fold_f.sum(axis=0))*f*100
plot_f = fold_f_n.mean(axis=1).values
funct_SHAP = pd.DataFrame({"SHAP_values" : plot_f}, index = DF_normalized.iloc[:, 222:].columns )
funct_SHAP['std']=fold_f_n.std(axis=1).values
funct_SHAP = funct_SHAP.sort_values(by="SHAP_values", ascending=False)
funct_SHAP

In [ ]:
all_m_s =  pd.concat([struct_SHAP,funct_SHAP])
all_for_SHAP = all_m_s.sort_values(by="SHAP_values", ascending=False)
all_for_SHAP

As a check, the sum of all the SHAP values must be equal to 100%

In [ ]:
all_for_SHAP.sum(axis='rows')

At this point, we have a dataframe that contains all the features from the most important to the less important. Now we select the scores above the 99th percentile of importance features selected by SHAP.

## Selection of 99th percentile of features importance

Since we cannot plot the feature importance for all the features, we select only the ones with an importance over the 99% of the importance values in the dataset. We will define two variables: ine with all the sorted feature importance and one with only the functional features. This will help in the visualization of the functional features.

In [ ]:
th99 = all_for_SHAP.iloc[:, 0].quantile(0.99)
th99

In [ ]:
ALL_SHAP_99 = all_for_SHAP[all_for_SHAP["SHAP_values"] >= th99]["SHAP_values"]
funct_SHAP_99 = funct_SHAP[funct_SHAP["SHAP_values"] >= th99]["SHAP_values"]

In [ ]:
important_funct_features_SHAP = funct_SHAP_99.index.astype('int').tolist()

In [ ]:
ax2 = plt.figure(figsize=(40, 50))
ax2 = all_for_SHAP[all_for_SHAP["SHAP_values"] >= th99]["SHAP_values"].plot(kind="barh", figsize=(10,10))
ax2.invert_yaxis()
plt.xlabel("mean(|SHAP value|)")
plt.show()

## Cohen d coeff

The Cohen's *d* coefficient is a statistical measure used to quantify the *effect size* between two groups, indicating the standardized difference between their means. It is commonly used in psychology, social sciences, and other fields to assess the difference between two sample groups.

### Formula

The formula for Cohen’s *d* is:
$$
d = \frac{\bar{X}_1 - \bar{X}_2}{s}
$$
where: $\bar{X}_1$ and $\bar{X}_2$ are the means of the two groups. $s$ is the pooled standard deviation of the two groups, calculated as:
$$
s = \sqrt{\frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2}}
$$
where: $n_1$ and $n_2$ are the sample sizes of the two groups. $s_1$ and $s_2$ are the standard deviations of the two groups.

### Interpretation
Cohen's *d* provides a way to interpret the magnitude of the difference, regardless of the scale of the data, making it easier to compare across studies. Common interpretations are:
- **0.2** - Small effect size
- **0.5** - Medium effect size
- **0.8** or higher - Large effect size

These are general guidelines, and the interpretation can vary by field. Cohen's *d* is particularly helpful because it puts the difference in a standardized context, allowing researchers to understand the size of an effect without being influenced by sample size alone.

In [ ]:
def Cohen_d(g1, g2, f):
    """Function to compute the Cohen's d Coefficient.

    g1: infered results by predictor 1
    g2: infered results by predictor 2
    f:  predicted class ?

    it returns the 'd' value of agreement
    """
    n1 = len(g1)                   # number of data in g1
    n2 = len(g2)                   # number of data in g2
    N = n1 + n2                    # total number of data
    Scores1 = g1[f].dropna()       # remove nan results
    Scores2 = g2[f].dropna()
    var1 = Scores1.var()           # compute the variance over the dataset
    var2 = Scores2.var()
    mean1 = Scores1.mean()         # compute the mean result
    mean2 = Scores2.mean()
    sp = (((n1 - 1)*var1 + (n2 - 1)*var2) / (N - 2))**0.5
    d = (mean1 - mean2) / sp
    return d

In [ ]:
controls = DF_normalized[DF_normalized.DX_GROUP==0]
ASD =  DF_normalized[DF_normalized.DX_GROUP==1]
list_f = all_m_s.iloc[:].index.tolist()
score_df = []
for item in list_f:
    score =  Cohen_d(ASD, controls, item)
    score_df.append(score)
    #print(item, score)
all_m_s['cohen']=score_df

In [ ]:
cohen_all = all_m_s.drop(['SHAP_values','std'], axis = 'columns')
cohen_funct = cohen_all[221:]
cohen_sorted = cohen_funct.sort_values(by="cohen", ascending=False)

In [ ]:
important_funct_features_COHEN_ASD = cohen_sorted[:20].index.astype('int').tolist()
important_funct_features_COHEN_TD = cohen_sorted[5233:].index.astype('int').tolist()

## Find region's coord in HO

Since functional features are not easy to be associated with the real meaning, i. e. it is not easy to understand the parts of the brain that are used to compute that feature, here we go back from the functional features to some images that helps in understanding their meaning.

In [ ]:
from nilearn import datasets, plotting

functional_names = pd.read_csv(path_to_data + 'functional_features.csv')
functional_names = functional_names.set_index('F')

Here, we reduce the nyumber of features to match the available color maps.

In [ ]:
reduction_features = important_funct_features_SHAP[:13]
perc_95 = functional_names.loc[reduction_features]
connection_list = functional_names.loc[reduction_features]
l1 = connection_list['r1'].to_list()
l2 = connection_list['r2'].to_list()
l = l1 + l2
sam_list = list(set(l)) # the redundant feature names are deleted
region_c = []
label_c = []

In [ ]:
#load HO atlas
atlas_ho = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')  #sub-maxprob-thr50-2mm - cortl-maxprob-thr25-2mm
atlas_file = atlas_ho.maps
labels = atlas_ho.labels[1:]
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_file)

In [ ]:
# Load labels for each atlas region

print(len(labels))

atlas_ho = datasets.fetch_atlas_harvard_oxford('cortl-maxprob-thr25-2mm')
atlas_file = atlas_ho.maps
# Load labels for each atlas region
atlas_labels = atlas_ho.labels[1:]
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_file)
print(len(labels))

for i, e in enumerate(atlas_labels):
  for j, n in enumerate(sam_list):
    if e == n:
      region_c.append(coordinates[i])
      label_c.append(e)

len(label_c)

## PLOT connectoma

In [ ]:
from matplotlib import colormaps as cm
from matplotlib.lines import Line2D

for i in range(0, len(label_c)):
    print("i, region_c[i],label_c[i]")
    print(i, region_c[i],label_c[i])

index_1 = []
for j, n in enumerate(l1):
    for i, e in enumerate(label_c):
        if n == e:
            index_1.append(i)

index_2=[]
for j, n in enumerate(l2):
    for i, e in enumerate(label_c):
        if n == e:
            index_2.append(i)

In [ ]:
mat =np.zeros((len(region_c), len(region_c)))

for index, (value1, value2) in enumerate(zip(index_1, index_2)):
    #print(index, value1 , value2)
    mat[value1][value2] = 1
mat

mat = mat + mat.T
coordinates = np.array(region_c)  # 3D coordinates of parcels

color_dict = {}
cmap = cm.get_cmap('tab20')

for rsn, c in zip(label_c, cmap.colors):
    color_dict[rsn] = tuple(c)

In [ ]:
node_color = []
for nw in label_c:
    node_color.append(color_dict[nw])

coords = coordinates
N = len(coords)

In [ ]:
plotting.plot_connectome(mat,
                         coords,
                         title='Most important features according to SHAP',
                         node_color=node_color,
                          display_mode="lyrz",
                         edge_kwargs = {"linewidth":1.7, "color": 'red'})
legend_elements = []
for k,v in color_dict.items():
    legend_elements.append(Line2D([0], [0], marker='o', color=v, label=k,
                          markerfacecolor=v, markersize=5))
# Create the figure
fig, ax = plt.subplots()
ax.axis("off")
ax.legend(handles=legend_elements, loc='center')

## Connettoma plot according to Cohen Values

### ASD > TD

In [ ]:
reduction_features = important_funct_features_COHEN_ASD
perc_95 = functional_names.loc[reduction_features]
connection_list = functional_names.loc[reduction_features]
l1 = connection_list['r1'].to_list()
l2 = connection_list['r2'].to_list()
l = l1 + l2
sam_list = list(set(l)) # the redundant feature names are deleted
region_c = []
label_c = []

In [ ]:
atlas_ho = datasets.fetch_atlas_harvard_oxford('cortl-maxprob-thr25-2mm')
atlas_file = atlas_ho.maps
# Load labels for each atlas region
atlas_labels = atlas_ho.labels[1:]
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_file)
print(len(labels))

for i, e in enumerate(atlas_labels):
  for j, n in enumerate(sam_list):
    if e == n:
      region_c.append(coordinates[i])
      label_c.append(e)

len(label_c)
for i in range(0, len(label_c)):
    print("i, region_c[i],label_c[i]")
    print(i, region_c[i],label_c[i])

index_1 = []
for j, n in enumerate(l1):
    for i, e in enumerate(label_c):
        if n == e:
            index_1.append(i)

index_2=[]
for j, n in enumerate(l2):
    for i, e in enumerate(label_c):
        if n == e:
            index_2.append(i)
mat =np.zeros((len(region_c), len(region_c)))

for index, (value1, value2) in enumerate(zip(index_1, index_2)):
    #print(index, value1 , value2)
    mat[value1][value2] = 1
mat

mat = mat + mat.T
coordinates = np.array(region_c)  # 3D coordinates of parcels

color_dict = {}
cmap = cm.get_cmap('tab20')

for rsn, c in zip(label_c, cmap.colors):
    color_dict[rsn] = tuple(c)
node_color = []
for nw in label_c:
    node_color.append(color_dict[nw])

coords = coordinates
N = len(coords)

In [ ]:
plotting.plot_connectome(mat,
                         coords,
                         title='ASD>TD',
                         node_color=node_color,
                          display_mode="lyrz",
                         edge_kwargs = {"linewidth":1.7, "color": 'red'})
legend_elements = []
for k,v in color_dict.items():
    legend_elements.append(Line2D([0], [0], marker='o', color=v, label=k,
                          markerfacecolor=v, markersize=5))
# Create the figure
fig, ax = plt.subplots()
ax.axis("off")
ax.legend(handles=legend_elements, loc='center')

### TD>ASD

In [ ]:
reduction_features = important_funct_features_COHEN_TD[:16]
perc_95 = functional_names.loc[reduction_features]
connection_list = functional_names.loc[reduction_features]
l1 = connection_list['r1'].to_list()
l2 = connection_list['r2'].to_list()
l = l1 + l2
sam_list = list(set(l)) # the redundant feature names are deleted
region_c = []
label_c = []

In [ ]:
atlas_ho = datasets.fetch_atlas_harvard_oxford('cortl-maxprob-thr25-2mm')
atlas_file = atlas_ho.maps
# Load labels for each atlas region
atlas_labels = atlas_ho.labels[1:]
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_file)

for i, e in enumerate(atlas_labels):
  for j, n in enumerate(sam_list):
    if e == n:
      region_c.append(coordinates[i])
      label_c.append(e)

len(label_c)
for i in range(0, len(label_c)):
    print("i, region_c[i],label_c[i]")
    print(i, region_c[i],label_c[i])

index_1 = []
for j, n in enumerate(l1):
    for i, e in enumerate(label_c):
        if n == e:
            index_1.append(i)

index_2=[]
for j, n in enumerate(l2):
    for i, e in enumerate(label_c):
        if n == e:
            index_2.append(i)
mat =np.zeros((len(region_c), len(region_c)))

for index, (value1, value2) in enumerate(zip(index_1, index_2)):
    #print(index, value1 , value2)
    mat[value1][value2] = 1
mat

mat = mat + mat.T
coordinates = np.array(region_c)  # 3D coordinates of parcels

color_dict = {}
cmap = cm.get_cmap('tab20')

for rsn, c in zip(label_c, cmap.colors):
    color_dict[rsn] = tuple(c)
node_color = []
for nw in label_c:
    node_color.append(color_dict[nw])

coords = coordinates
N = len(coords)

In [ ]:
plotting.plot_connectome(mat,
                         coords,
                         title='TD>ASD',
                         node_color=node_color,
                          display_mode="lyrz",
                         edge_kwargs = {"linewidth":1.7, "color": 'blue'})
legend_elements = []
for k,v in color_dict.items():
    legend_elements.append(Line2D([0], [0], marker='o', color=v, label=k,
                          markerfacecolor=v, markersize=5))
# Create the figure
fig, ax = plt.subplots()
ax.axis("off")
ax.legend(handles=legend_elements, loc='center')